### Import libraries 

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.io as sio
import os
import matplotlib.pyplot as plt 
import itertools
from itertools import groupby, combinations
from all_observables import *
from discrepancy_measures import *
from test_retest_reliability import *

### Load data

In [2]:
file_path = "/Users/sislam/Documents/fMRI_Microstate/fmri_state_transition_dynamics/sample_data/"
# First participant and first session data 
df = pd.read_csv(file_path+"sample_data_participant1_session1.csv")

### Assign the cluster label to each time point and compute the centroid of each cluster using K-means clustering.

In [3]:
data = np.array(df)
kmeans_labels, kmeans_centroids = Kmeans(data = data, num_clusters = 4)

### Getting cluster labels, GEV and all observables of the first session data of first participant.

In [4]:
labels, GEV, observables = lav_observ(data = data,  method = Kmeans, num_clusters = 4)

### Discrepancy measures
- **Dissimilarity between centroid position**
- **Total Variation of coverage time** 
- **Total Variation of frequency** 
- **Total Variation of average lifespan** 
- **Frobenius distance of two transition probability matrices** 

Two compare the observables between two sessions from the same participant or from different pariticapnt, we need all observables from all sessions. 

In [5]:
# all observables from all sessions
all_results = [[lav_observ(data = np.array(pd.read_csv(file_path+"sample_data_participant"+str(p)+"_session"+str(s)+".csv")),  
           method = Kmeans, num_clusters = 4)[2] for s in range(1,3)] for p in range(1,3)]

# reproducibility of five observables within subjects and between subjects.
within_sub = reproducibility(all_results = np.array(all_results), distance = "cosine", WP = True, BP=None)
between_sub = reproducibility(all_results = np.array(all_results), distance = "cosine", WP = None, BP = True)

### Permutation test
We hypothesize that the state-transition dynamics estimated
from fMRI data is more consistent between different sessions of
the same participant than between different participants. To test
this hypothesis, we compare the dissimilarity between two sessions
originating from the same participant and the dissimilarity
between two sessions originating from different participants.For each observable,
we compare the within-participant dissimilarity and
between-participant dissimilarity using the normalized distance
ND combined with the permutation test.

In [9]:
ND_empirical, ND_permuted, p_value = permuted_ND(N = 10, 
                                                 observable_results = np.array(all_results), 
                                                 distance = "cosine")

***If someone would like to get only ND value for the five observables from empirical data then the following command can be used.***

In [8]:
ND_empirical = ND_value(observable_results = np.array(all_results), distance = "cosine")